In [23]:
!pip install -q langchain langchain-openai langgraph python-dotenv openai pydub faiss-cpu langchain-community

In [24]:
import os
from pathlib import Path
from typing import List, Dict, Optional, TypedDict

from dotenv import load_dotenv
from IPython.display import Audio, display
from google.colab import files

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import StateGraph, END

from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

from openai import OpenAI

load_dotenv()

# If key not set in environment, ask for it
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API key: ").strip()

if os.getenv("OPENAI_API_KEY"):
    print("✅ OPENAI_API_KEY loaded.")
else:
    print("❌ OPENAI_API_KEY missing.")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("✅ OpenAI client ready.")

MODEL_NAME = "gpt-4o-mini"  # change to "gpt-4.1" if you have it


✅ OPENAI_API_KEY loaded.
✅ OpenAI client ready.


In [25]:
product_docs = [
    Document(
        page_content="""
Nako Treasury is a B2B investment intelligence and treasury management platform.
It helps small and mid-sized businesses invest idle cash safely and intelligently.

Core capabilities:
- Real-time portfolio analytics (performance, risk, drawdown).
- Risk scoring and alerts based on market conditions.
- Cashflow-aware recommendations that avoid over-investing working capital.
- Automated allocation across low-risk instruments (T-bills, MMFs, short-term bonds).
- Compliance-grade reporting for auditors, board, and investors.
- Scenario forecasting (what-if analysis, stress tests).
- Competitor benchmarking vs traditional bank deposits and platforms like Revolut Business and Wise.

Value proposition:
"Turn idle cash into a safe, intelligent asset — with full transparency and real-time risk protection."
""",
        metadata={"source": "product_overview"}
    ),
    Document(
        page_content="""
Common objections for Nako Treasury:

1. "Is this safe?"
   - Nako focuses on diversified, low-risk instruments.
   - We provide transparent risk scoring and worst-case scenario views.
2. "What if markets drop?"
   - Portfolios are stress-tested and optimized for capital preservation, not speculation.
   - Risk alerts and rebalancing algorithms reduce exposure during volatile periods.
3. "We don’t trust automated platforms."
   - Nako combines automation with human-audited models and clear governance policies.
   - Every recommendation comes with an explanation and rationale.
4. "How liquid are the funds?"
   - Most instruments are highly liquid, and we model liquidity needs based on past cashflow.
5. "We already use solutions like Revolut or our bank."
   - Nako focuses specifically on treasury analytics, risk insight, and optimization on top of existing accounts.
""",
        metadata={"source": "objections"}
    ),
]

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(product_docs, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print("✅ Vector store built with", len(product_docs), "documents.")


✅ Vector store built with 2 documents.


In [26]:
def upload_audio_file() -> str:
    """
    Opens a file upload widget in Colab, lets you pick an audio file,
    and returns the local filename.
    """
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("No file uploaded.")
    filename = next(iter(uploaded.keys()))
    print(f"📁 Uploaded:", filename)
    return filename


In [27]:
def transcribe_audio(path: str) -> str:
    """
    Transcribe an audio file using OpenAI's Whisper (whisper-1).
    """
    print(f"🎙️ Transcribing {path} ...")
    with open(path, "rb") as f:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=f,
        )
    text = transcript.text.strip()
    print("📝 Transcription:", text)
    return text


In [28]:
def speak_text(text: str, out_path: str = "reply.mp3", voice: str = "alloy"):
    """
    Convert text to speech using OpenAI TTS and play it in the notebook.
    """
    print("🔊 Generating audio reply...")
    response = client.audio.speech.create(
        model="tts-1",
        voice=voice,
        input=text,
    )
    out_file = Path(out_path)
    response.stream_to_file(out_file)
    print(f"💾 Saved audio to {out_file}")

    display(Audio(str(out_file), autoplay=True))


In [29]:
class SimulationState(TypedDict):
    messages: List[BaseMessage]       # conversation history
    persona: str                      # "founder" | "finance_manager" | "cfo"
    turn: int                         # salesperson turns so far
    evaluation: Dict[str, float]      # scores from evaluator
    coaching: str                     # coaching text
    next: str                         # which node to go to next
    # Scenario-related fields:
    difficulty: str                   # "easy" | "medium" | "hard"
    context: str                      # short business context
    start_mode: str                   # "salesperson_starts" | "customer_starts"
    starting_objection: Optional[str] # first customer line if they start
    target_skills: List[str]          # skills to focus on
    scenario_description: str         # human-readable scenario description


def persona_description(persona: str) -> str:
    if persona == "founder":
        return (
            "You are a startup founder. You are friendly, move fast, and are skeptical about risk. "
            "You care mostly about runway, not losing money, and keeping things simple."
        )
    elif persona == "finance_manager":
        return (
            "You are an SMB finance manager. You are analytical and cautious. "
            "You care about compliance, controls, liquidity, and making sure there is always enough cash "
            "for payroll, vendors, and operating expenses."
        )
    elif persona == "cfo":
        return (
            "You are a CFO of a mid-sized business. You are highly data-driven and detail-oriented. "
            "You care about risk-adjusted returns, drawdown, governance, reporting, and how this fits into "
            "your overall treasury policy."
        )
    else:
        return "You are a generic business stakeholder evaluating Nako Treasury."


In [30]:
import re
import json

def generate_scenario_config(user_description: str) -> Dict:
    """
    Use LLM to turn a natural language scenario description into
    a structured JSON scenario config.
    """
    prompt = f"""
You are a scenario generation agent for a sales training simulation about Nako Treasury,
a B2B treasury and idle-cash investment platform.

User description of the scenario:
\"\"\"{user_description}\"\"\"

Your job is to generate a JSON config with the following fields:

- persona: one of ["founder", "finance_manager", "cfo"]
- difficulty: one of ["easy", "medium", "hard"]
- start_mode: one of ["salesperson_starts", "customer_starts"]
- context: a short 1–2 sentence description of the business context
- starting_objection: a short sentence the customer might open with
  (if start_mode is "customer_starts"; otherwise can be null or empty)
- target_skills: 3–5 skills chosen from:
  ["clarity", "empathy", "objection_handling", "product_knowledge", "relevance", "confidence", "active_listening", "closing"]
- scenario_description: 2–3 sentences describing the scenario in natural language.

Return STRICT JSON in this format (no explanation, no markdown):

{{
  "persona": "...",
  "difficulty": "...",
  "start_mode": "...",
  "context": "...",
  "starting_objection": "...",
  "target_skills": ["...", "..."],
  "scenario_description": "..."
}}
"""

    llm = ChatOpenAI(model=MODEL_NAME, temperature=0.5)
    response = llm.invoke([HumanMessage(content=prompt)]).content

    try:
        config = json.loads(response)
    except Exception:
        m = re.search(r"\{.*\}", response, re.DOTALL)
        if not m:
            raise ValueError("Could not parse scenario config JSON.")
        config = json.loads(m.group(0))

    persona = config.get("persona", "founder")
    if persona not in ["founder", "finance_manager", "cfo"]:
        persona = "founder"

    difficulty = config.get("difficulty", "medium")
    if difficulty not in ["easy", "medium", "hard"]:
        difficulty = "medium"

    start_mode = config.get("start_mode", "salesperson_starts")
    if start_mode not in ["salesperson_starts", "customer_starts"]:
        start_mode = "salesperson_starts"

    context = config.get("context", "")
    starting_objection = config.get("starting_objection") or ""
    target_skills = config.get("target_skills", ["clarity", "objection_handling"])
    scenario_description = config.get("scenario_description", user_description)

    return {
        "persona": persona,
        "difficulty": difficulty,
        "start_mode": start_mode,
        "context": context,
        "starting_objection": starting_objection,
        "target_skills": target_skills,
        "scenario_description": scenario_description,
    }


def init_state_from_scenario(user_description: str) -> SimulationState:
    """
    High-level helper: generate scenario config and build initial simulation state.
    """
    config = generate_scenario_config(user_description)

    state: SimulationState = {
        "messages": [],
        "persona": config["persona"],
        "turn": 0,
        "evaluation": {},
        "coaching": "",
        "next": "dialogue",
        "difficulty": config["difficulty"],
        "context": config["context"],
        "start_mode": config["start_mode"],
        "starting_objection": config["starting_objection"],
        "target_skills": config["target_skills"],
        "scenario_description": config["scenario_description"],
    }

    return state


In [31]:
test_config = generate_scenario_config(
    "Hard scenario: CFO worried about liquidity and skeptical of automated treasury tools."
)
test_config


{'persona': 'cfo',
 'difficulty': 'hard',
 'start_mode': 'customer_starts',
 'context': 'The CFO of a mid-sized manufacturing company is concerned about maintaining liquidity while exploring investment options for idle cash.',
 'starting_objection': "I'm not convinced that automated treasury tools can effectively manage our liquidity needs.",
 'target_skills': ['objection_handling',
  'empathy',
  'product_knowledge',
  'active_listening',
  'confidence'],
 'scenario_description': "In this scenario, the CFO expresses skepticism about the effectiveness of automated treasury tools in managing liquidity. The salesperson must address the CFO's concerns while demonstrating the value of Nako Treasury's platform in optimizing cash management and ensuring liquidity."}

In [32]:
def dialogue_node(state: SimulationState) -> SimulationState:
    """Persona/customer agent that responds based on RAG + persona + context."""
    messages = state["messages"]
    persona = state["persona"]
    context_text = state.get("context", "")

    # Last human message = salesperson's last response
    last_user = None
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            last_user = msg.content
            break

    query = last_user or "Ask a relevant question about Nako Treasury."

    docs = retriever.invoke(query)
    kb_context = "\n\n".join([d.page_content for d in docs])

    system_prompt = f"""
You are the CUSTOMER in a sales simulation about Nako Treasury, a B2B treasury and investment
intelligence platform.

Persona:
{persona_description(persona)}

Business context:
{context_text}

Your job:
- Respond as the customer.
- Raise realistic questions and objections about safety, risk, liquidity, existing solutions, or complexity.
- Keep your responses human, natural, and conversational.
- Base your knowledge primarily on the context and Nako Treasury docs below.

Nako Treasury docs:
{kb_context}
"""

    llm = ChatOpenAI(model=MODEL_NAME, temperature=0.7)
    convo = [SystemMessage(content=system_prompt)] + messages
    customer_reply = llm.invoke(convo)

    new_messages = messages + [customer_reply]
    return {
        **state,
        "messages": new_messages,
        "turn": state.get("turn", 0) + 1,
        "next": "evaluator",
    }


In [33]:
def evaluator_node(state: SimulationState) -> SimulationState:
    """Evaluate the salesperson's last answer using skill-aware rubric."""
    messages = state["messages"]
    difficulty = state.get("difficulty", "medium")
    target_skills = state.get("target_skills", [])

    last_sales_msg = None
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            last_sales_msg = msg.content
            break

    last_customer_msg = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage):
            last_customer_msg = msg.content
            break

    eval_prompt = f"""
You are an evaluation agent for a sales training simulation for Nako Treasury.

Scenario difficulty: {difficulty}
Target skills for this scenario: {target_skills}

Score the salesperson's last response against this rubric (0–10 each):
- clarity
- empathy
- objection_handling
- product_knowledge
- relevance
- confidence

Higher difficulty means you should be stricter with high scores (9–10).

Customer said:
\"\"\"{last_customer_msg}\"\"\"

Salesperson responded:
\"\"\"{last_sales_msg}\"\"\"

Return STRICT JSON of the form:
{{
  "clarity": float,
  "empathy": float,
  "objection_handling": float,
  "product_knowledge": float,
  "relevance": float,
  "confidence": float
}}
No explanation, just JSON.
"""

    llm = ChatOpenAI(model=MODEL_NAME, temperature=0.0)
    response = llm.invoke([HumanMessage(content=eval_prompt)])
    text = response.content

    scores: Dict[str, float] = {}
    try:
        scores = json.loads(text)
    except Exception:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            try:
                scores = json.loads(match.group(0))
            except Exception:
                pass

    default_scores = {
        "clarity": 5.0,
        "empathy": 5.0,
        "objection_handling": 5.0,
        "product_knowledge": 5.0,
        "relevance": 5.0,
        "confidence": 5.0,
    }
    default_scores.update({k: float(v) for k, v in scores.items() if k in default_scores})

    return {
        **state,
        "evaluation": default_scores,
        "next": "coach",
    }


In [34]:
def coach_node(state: SimulationState) -> SimulationState:
    """Generate coaching feedback based on the evaluation scores."""
    scores = state.get("evaluation", {})
    messages = state["messages"]
    target_skills = state.get("target_skills", [])

    last_sales_msg = None
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            last_sales_msg = msg.content
            break

    coaching_prompt = f"""
You are a coaching agent in a sales training simulation for Nako Treasury.

Scores:
{json.dumps(scores, indent=2)}

Target skills for this scenario: {target_skills}

Salesperson's last response:
\"\"\"{last_sales_msg}\"\"\"

Give:
1. One short positive reinforcement sentence.
2. 2-3 specific, actionable coaching tips, with special focus on the weaker scores and the target skills.
3. Keep it under 6 bullet points.

Respond in plain text.
"""

    llm = ChatOpenAI(model=MODEL_NAME, temperature=0.7)
    response = llm.invoke([HumanMessage(content=coaching_prompt)])

    return {
        **state,
        "coaching": response.content,
        "next": "safety",
    }


In [35]:
def safety_node(state: SimulationState) -> SimulationState:
    """Very simple safety/compliance checker."""
    messages = state["messages"]

    last_sales_msg = None
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            last_sales_msg = msg.content
            break

    safety_prompt = f"""
You are a compliance and safety agent for a B2B financial product (Nako Treasury).

Review the salesperson's last message for:
- unrealistic guarantees
- promises of no risk
- misleading performance claims
- anything that would be non-compliant in a regulated financial context.

If it's all fine, respond with: "SAFE".
If not, respond with: "UNSAFE: <one sentence explanation>".
"""

    llm = ChatOpenAI(model=MODEL_NAME, temperature=0.0)
    response = llm.invoke([HumanMessage(content=safety_prompt)])

    safety_flag = response.content.strip()

    coaching = state.get("coaching", "")
    coaching += f"\n\n[Safety check]: {safety_flag}"

    return {
        **state,
        "coaching": coaching,
        "next": "orchestrator",
    }


In [37]:
MAX_TURNS = 3  # number of salesperson turns in the simulation

def orchestrator_node(state: SimulationState) -> SimulationState:
    """
    Simple orchestrator that decides whether to continue the simulation
    or end it, based on how many salesperson turns have happened so far.
    """
    turn = state.get("turn", 0)
    if turn >= MAX_TURNS:
        # End the simulation
        return {**state, "next": "end"}
    else:
        # Continue with the dialogue node
        return {**state, "next": "dialogue"}


In [38]:
def output_node(state: SimulationState) -> SimulationState:
    """Final node – just returns state."""
    return {**state, "next": "end"}


def build_simulation_graph():
    workflow = StateGraph(SimulationState)

    workflow.add_node("orchestrator", orchestrator_node)
    workflow.add_node("dialogue", dialogue_node)
    workflow.add_node("evaluator", evaluator_node)
    workflow.add_node("coach", coach_node)
    workflow.add_node("safety", safety_node)
    workflow.add_node("output", output_node)

    def route_from_orchestrator(state: SimulationState) -> str:
        return state["next"]

    workflow.add_conditional_edges(
        "orchestrator",
        route_from_orchestrator,
        {"dialogue": "dialogue", "end": "output"},
    )

    workflow.add_edge("dialogue", "evaluator")
    workflow.add_edge("evaluator", "coach")
    workflow.add_edge("coach", "safety")
    workflow.add_edge("safety", "orchestrator")

    workflow.set_entry_point("orchestrator")

    app = workflow.compile()
    return app

simulation_app = build_simulation_graph()
print("✅ Simulation graph built with full multi-agent pipeline.")


✅ Simulation graph built with full multi-agent pipeline.


In [39]:
def run_voice_scenario_simulation(description: str, max_turns: int = 3, tts_voice: str = "alloy") -> SimulationState:
    """
    Voice-enabled version of the scenario simulation:
    - Generate scenario from natural language description.
    - If customer starts, play their starting objection via TTS.
    - For each turn:
        - You upload an audio file as salesperson.
        - Audio is transcribed with Whisper.
        - Multi-agent sim runs.
        - Customer reply is spoken via TTS.
    """
    global MAX_TURNS
    MAX_TURNS = max_turns

    state = init_state_from_scenario(description)

    print("=== Generated Scenario ===")
    print("Persona:           ", state["persona"])
    print("Difficulty:        ", state["difficulty"])
    print("Start mode:        ", state["start_mode"])
    print("Context:           ", state["context"])
    print("Target skills:     ", state["target_skills"])
    print("Description:       ", state["scenario_description"])
    print("Starting objection:", state["starting_objection"] or "(none)")
    print("==========================\n")

    # If the customer starts, inject their starting objection as first AI message
    if state["start_mode"] == "customer_starts" and state["starting_objection"]:
        first_ai = AIMessage(content=state["starting_objection"])
        state["messages"].append(first_ai)
        print("[Customer starts]:")
        print(state["starting_objection"])
        speak_text(state["starting_objection"], out_path="customer_start.mp3", voice=tts_voice)
        print()

    print("🎧 You are the salesperson. The AI is the customer.\n")

    for i in range(MAX_TURNS):
        print(f"[Your turn {i+1}] Record your response as audio (e.g., voice memo), then upload the file here.")
        audio_path = upload_audio_file()
        transcript = transcribe_audio(audio_path)

        if not transcript:
            print("No speech detected. Ending simulation.")
            break

        state["messages"].append(HumanMessage(content=transcript))
        state = simulation_app.invoke(state)

        # Get last AI (customer) reply
        last_ai = None
        for msg in reversed(state["messages"]):
            if isinstance(msg, AIMessage):
                last_ai = msg.content
                break

        print("\n----------------------")
        print("[Customer persona]:")
        print(last_ai)

        # Speak customer reply
        speak_text(last_ai, out_path=f"customer_reply_turn_{i+1}.mp3", voice=tts_voice)

        # Show scores + coaching
        print("\n[Scores]:")
        for k, v in state.get("evaluation", {}).items():
            print(f"  {k}: {v:.1f}")

        print("\n[Coaching]:")
        print(state.get("coaching", "No coaching yet."))
        print("----------------------\n")

    print("✅ Voice simulation complete.")
    return state


In [40]:
desc_voice = "Medium scenario: founder worried about losing money if markets drop, but curious about using Nako Treasury for idle cash."
final_voice_state = run_voice_scenario_simulation(desc_voice, max_turns=2, tts_voice="alloy")


=== Generated Scenario ===
Persona:            founder
Difficulty:         medium
Start mode:         customer_starts
Context:            The founder is managing a startup that has accumulated idle cash but is concerned about potential market downturns affecting their investments.
Target skills:      ['empathy', 'objection_handling', 'product_knowledge', 'active_listening', 'confidence']
Description:        In this scenario, the founder expresses their concerns about market volatility and the risks associated with investing idle cash. They are curious about how Nako Treasury can help them manage their cash effectively while minimizing risk. The salesperson needs to address these concerns while demonstrating the value of the platform.
Starting objection: I'm worried about losing money if the markets drop.

[Customer starts]:
I'm worried about losing money if the markets drop.
🔊 Generating audio reply...
💾 Saved audio to customer_start.mp3


/tmp/ipython-input-458821402.py:12: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(out_file)



🎧 You are the salesperson. The AI is the customer.

[Your turn 1] Record your response as audio (e.g., voice memo), then upload the file here.


Saving Voice_1.ogg to Voice_1.ogg
📁 Uploaded: Voice_1.ogg
🎙️ Transcribing Voice_1.ogg ...
📝 Transcription: So as a salesperson, what should I do to? Limit the risk.

----------------------
[Customer persona]:
I see that you focus on low-risk instruments, but how do I know that these instruments are really safe? What kind of transparency do you provide regarding the risk involved? I want to feel confident that my funds won't be at risk.
🔊 Generating audio reply...
💾 Saved audio to customer_reply_turn_1.mp3


/tmp/ipython-input-458821402.py:12: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(out_file)



[Scores]:
  clarity: 2.0
  empathy: 1.0
  objection_handling: 2.0
  product_knowledge: 1.0
  relevance: 2.0
  confidence: 2.0

[Coaching]:
1. Great effort in focusing on risk management; it shows you're thinking about your client's needs!

2. 
   - **Empathy**: Try to connect with your client by acknowledging their concerns. For example, say something like, "I understand that managing risk is important to you."
   - **Objection Handling**: Practice responding to potential objections by preparing specific counterarguments that address common client concerns. This will help you feel more confident when objections arise.
   - **Product Knowledge**: Take time to study the features and benefits of your products in-depth. This will allow you to confidently answer questions and highlight how your products can specifically mitigate risks for clients. 

3. Remember to actively listen to your clients during conversations; this will help you tailor your responses and demonstrate that you value t

Saving Voice 2.ogg to Voice 2.ogg
📁 Uploaded: Voice 2.ogg
🎙️ Transcribing Voice 2.ogg ...
📝 Transcription: That's a completely fair question. The first thing we do is show you a clear risk score for every instrument, including worst case scenarios during market stress. You'll also see full transparency around what's in the portfolio, why each instrument was selected and how each one behaves in a downturn. Our priority is capital preservation. So you always know exactly where your cash is allocated and what the risk looks like.

----------------------
[Customer persona]:
I see that you focus on low-risk instruments, but how do I know that these instruments are really safe? What kind of transparency do you provide regarding the risk involved? I want to feel confident that my funds won't be at risk.
🔊 Generating audio reply...
💾 Saved audio to customer_reply_turn_2.mp3


/tmp/ipython-input-458821402.py:12: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(out_file)



[Scores]:
  clarity: 2.0
  empathy: 1.0
  objection_handling: 2.0
  product_knowledge: 1.0
  relevance: 2.0
  confidence: 2.0

[Coaching]:
1. Great effort in focusing on risk management; it shows you're thinking about your client's needs!

2. 
   - **Empathy**: Try to connect with your client by acknowledging their concerns. For example, say something like, "I understand that managing risk is important to you."
   - **Objection Handling**: Practice responding to potential objections by preparing specific counterarguments that address common client concerns. This will help you feel more confident when objections arise.
   - **Product Knowledge**: Take time to study the features and benefits of your products in-depth. This will allow you to confidently answer questions and highlight how your products can specifically mitigate risks for clients. 

3. Remember to actively listen to your clients during conversations; this will help you tailor your responses and demonstrate that you value t